# Импорт библиотек

In [152]:
import pandas as pd
import numpy as np

from collections import defaultdict
from itertools import product

import re

import requests

import wikipedia
wikipedia.set_lang('ru')

from sklearn.feature_extraction.text import TfidfVectorizer

In [270]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

In [252]:
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [6]:
# возьмем обработанный датасет из предыдущей лабораторной работы
df = pd.read_csv('LW4/jobs.csv')[['text', 'normalized_text']]

df['normalized_text'] = df['normalized_text'].map(lambda tokens: ' '.join(eval(tokens)))

# удалим пустые документы
df = df[df['normalized_text'].map(lambda tokens: len(tokens)) > 0].reset_index(drop=True)
df

,text,normalized_text
0,"ребят, привет! заранее извиняюсь, если пишу не...",ребята привет заранее извиняться писать нужный...
1,привет!\n*позиция:* team lead (data science)\n...,привет позиция компания лаборатория искусствен...
2,всем привет!\nищу в свою команду big data инже...,весь привет искать команда инженер компания пл...
3,"эта вакансия для тех, кто хочет *решать соревн...",этот вакансия тот хотеть решать соревнование р...
4,*компания:* сбербанк\n*вакансия:* data scienti...,компания сбербанк вакансия город москва вилка ...
...,...,...
3740,дружественная нам исследовательская группа нан...,дружественный мы исследовательский группа нани...
3741,"мопед не мой, так что все вопросы по указанном...",мопед вопрос указанный коллега попросить отпра...
3742,"отношения никакого к университету не имею, но ...",отношение никакой университет иметь решить под...
3743,всем привет!\nу нас в гренобльском офисе crite...,весь привет гренобльский офис открыться позици...


# Обработка текста

In [8]:
def check_spelling(text: str):
    """
    Исправляет опечатки слов в тексте с помощью Yandex Spellchecker API
    param text: текст, в котором нужно исправить опечатки
    """
    
    domain = "https://speller.yandex.net/services/spellservice.json"
    words = text.split()
    if len(words) == 1:
        request = requests.get(domain + "/checkText?text=" + words[0])
        if request.json():
            return request.json()[0]["word"], request.json()[0]["s"]
        else:
            return None
    
    elif len(words) > 1:
        words = "+".join(words)
        request = requests.get(domain + "/checkText?text=" + words)
        if request.json():
            response = [(i["word"], i["s"]) for i in request.json()]
            return response
        else:
            return None
    return None


def edit_spelling(query: str) -> str:
    """
    Редактирует текст, исправляя в нем опечатки. В случае если не удалось найти в корпусе Yandex заданное слово,
    оно не изменяется.
    param query: текст для редактиврования
    """
    
    edited_words = []
    words = query.split()
    for word in words:
        edited_word = check_spelling(word)
        edited_words.append(word if edited_word is None else edited_word[1][0])
    return ' '.join(edited_words)

In [116]:
def text_to_wordlist(text: str) -> list:
    """
    Преобразует текст в список слов, удаляя в нем символы пунктуации, цифры и другие лишние символы
    param text: текст для преобразования
    """
    
    text = re.sub(r"http[s]?://(?:[а-яА-Я]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", text)
    text = re.sub("[^а-яА-Я]"," ", text)
    return text.lower().split()


def remove_stopwords(words: list) -> list:
    """
    Удаляет стоп-слова
    param words: список слов, в которых нужно удалить стоп-слова
    """
    
    return [w for w in words if not w in stopwords.words('russian')]



def process_text(text: str) -> str:
    """
    Токенизация текста с последующим удалением стоп-слов
    param text: текст
    """
    
    raw_texts = tokenizer.tokenize(remove_accents(text).strip())
    texts = []
    for raw_text in raw_texts:
        if len(raw_text) > 0:
            texts.append(remove_stopwords(text_to_wordlist(raw_text)))
    return [item for sublist in texts for item in sublist]



def normalize(token: str) -> str:
    """
    Нормализация токена
    param token: токен
    """
    
    return morph.parse(token)[0].normal_form

In [147]:
def process_query(query: str, correct_spelling=True) -> str:
    """
    Редактирование запроса. Включает в себя следующие шаги:
    1. Исправление опечаток
    2. Токенизация
    3. Удаление стоп-слов
    4. Нормализация
    
    param query: запрос
    param correct_spelling: нужно ли исправлять опечатки в тексе
    """
    
    if correct_spelling:
        query = edit_spelling(query)
    return ' '.join(list(map(lambda word: normalize(word), process_text(query))))

In [11]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
morph = pymorphy2.MorphAnalyzer()

# Информационный поиск и ранжирование по документам

In [7]:
def cosine(a: list, b: list) -> np.float64:
    """
    Считает косинусное расстояние между векторами
    param a, b: векторы
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [53]:
def recommend(corpus: pd.Series, query: str, top_n: int = 10) -> list:
    """
    Возвращает наиболее похожие тексты на основании запроса
    param df: датафрейм, содержащий тексты
    param query: текст, для которого нужно сделать рекомендацию. Возвращает список индексов в датасете df
    param top_n: количество рекомендуемых текстов
    """
    
    query = process_query(query)
    
    vectorizer = TfidfVectorizer()
    vectorized_texts = list(vectorizer.fit_transform(list(corpus) + [query]).toarray())
    vectorized_corpus = vectorized_texts[:-1]
    vectorized_query = vectorized_texts[-1]
    
    recommendations = sorted(enumerate(vectorized_corpus), key=lambda text: cosine(text[1], vectorized_query))
    return [index for index, text in recommendations[-top_n:]]

## Пример запроса

In [272]:
query = 'Привет! Нам нушен в стартап классныъ инженер для выпалненнния сложных задач'
recommendation_indices = recommend(df['normalized_text'], query, top_n=15)
df.loc[recommendation_indices]

,text,normalized_text
972,всем привет! у нас в классной команде яндекса ...,весь привет классный команда яндекс вакансия о...
3138,всем привет! ищем к нам в jetlore опытных и на...,весь привет искать мы опытный начинающий разра...
1464,"привет, :ods:!\n\n*компания:* <https://sixfold...",привет компания позиция уровень мидло вилка ло...
3030,• потому что у нас клевая опытная команда (вып...,клёвый опытный команда выпускник преподавать к...
356,*вакансия*: data scientist / cto с потенциалом...,вакансия потенциал стать компания стартап лока...
94,всем привет!\n\nк нам в команду требуется *sen...,весь привет мы команда требоваться расположить...
483,в один небольшой китайский стартап (500к челов...,небольшой китайский стартап человек технология...
460,"всем привет!\nпитер, нужен nlp - специалист!\n...",весь привет питер нужный специалист ссылка вак...
333,всем привет!\nмы в ringlabs в киев очень ищем ...,весь привет киев очень искать наш отдел задача...
1712,data scientist / gismart\n<http://gismart.com>...,искать человек разный уровень огромный опыт ра...


# Рекомендация и ранжирование статей из Википедии

In [181]:
def frequency_score(text_1: str, text_2: str) -> float:
    """
    Считает частоту встречаемости слов в двух текстах
    Возвращает нормализованное значение от 0 до 1
    param text_1, text_2: тексты
    """
    
    words_1, words_2 = text_1.split(), text_2.split()
    return sum(map(lambda pair: pair[0] == pair[1], product(words_1, words_2))) / (len(words_1) * len(words_2))

In [237]:
def find_all_occurences(element: any, lst: list) -> list:
    """
    Находит все индексы вхождения элемента в список
    param element: элемент
    param lst: список 
    """
    result = []
    offset = -1
    while True:
        try:
            offset = lst.index(element, offset + 1)
        except ValueError:
            return result
        result.append(offset)


def distance_score(query: str, text: str) -> float:
    """
    Для всех пар слов в запросе считает среднее ближайшее расстояние между этими словами в тексте.
    Возвращает нормализованное значение от 0 до 1
    param query: запрос
    param text: текст
    """
    
    scores = []
    text_words = text.split()
    query_words = query.split()
    query_word_pairs = list(product(query_words, query_words))
    
    for previous_word, next_word in query_word_pairs:
        previous_indices = find_all_occurences(previous_word, text_words)
        next_indices = find_all_occurences(next_word, text_words)
        
        if not previous_indices or not next_indices:
            min_distance = len(text_words)
        else:
            min_distance = min(map(lambda indices: abs(indices[0] - indices[1]), product(previous_indices, next_indices)))
        scores.append(1 - (min_distance) / len(text_words))
    
    return np.mean(scores)

In [207]:
def position_score(query: str, text: str) -> float:
    """
    Для каждого слова в запросе считает, насколько близко оно находится от начала текста
    Возвращает нормализованное значение от 0 до 1
    param query: запрос
    param text: текст
    """
    
    scores = []
    query_words = query.split()
    text_words = text.split()
    for query_word in query_words:
        if query_word not in text_words:
            scores.append(0)
        else:
            scores.append(1 - text_words.index(query_word) / len(text_words))
            
    return np.mean(scores)

In [247]:
def recommend_wikipedia_articles(query: str) -> pd.DataFrame:
    """
    Рекомендует статьи из Википедии по словам из запроса
    param query: запрос
    """
    
    query = process_text(edit_spelling(query))
    titles = []
    for word in query:
        titles += wikipedia.search(word)
        
    articles = pd.DataFrame(index=range(len(titles)), columns=['title', 'url', 'text'])
    for i, title in enumerate(titles):
        try:
            articles.loc[i, 'title'] = title
            articles.loc[i, 'url'] = wikipedia.page(title).url
            articles.loc[i, 'text'] = wikipedia.summary(title)
        except:
            continue
            
    articles = articles[articles['text'] != ''].dropna(how='any').reset_index(drop=True)
    return articles

In [268]:
def range_articles(query: str, articles: pd.Series) -> pd.DataFrame:
    """
    Ранжирует статьи на основе запроса. Учитывает следующие типы рангов:
    1. Частота слов
    2. Расстояние между словами
    3. Расположение в документе
    Каждой статье присваивается ранг, равный среднему из посчитанных рангов
    param query: запрос
    param articles: список текстов (статей)
    """
    
    query = ' '.join(process_text(edit_spelling(query)))
    ranks = pd.DataFrame(index=range(len(articles)),
                         columns=['article', 'frequency_rank', 'distance_rank', 'position_rank'])
    
    for i, article in enumerate(articles):
        ranks.loc[i, 'article'] = article
        article = ' '.join(process_text(article))
        
        ranks.loc[i, 'frequency_rank'] = frequency_score(query, article)
        ranks.loc[i, 'distance_rank'] = distance_score(query, article)
        ranks.loc[i, 'position_rank'] = position_score(query, article)
    
    ranks['rank'] = (ranks['frequency_rank'] + ranks['distance_rank'] + ranks['position_rank']) / 3
    return ranks.sort_values(by='rank', ascending=False)

## Пример запроса

In [259]:
query = 'Разрабочики баз даных умныые'

In [260]:
# сделаем рекомендацию для запроса
articles = recommend_wikipedia_articles(query)
articles

,title,url,text
0,Разработчик,https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%...,"Разрабо́тчик — специалист, занимающийся разраб..."
1,Разработчик видеоигр,https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%...,Разработчик видеоигр — это разработчик програм...
2,Rebellion (разработчик),https://ru.wikipedia.org/wiki/Rebellion_(%D1%8...,Rebellion — британская компания по разработке ...
3,Roblox,https://ru.wikipedia.org/wiki/Roblox,Roblox — многопользовательская онлайн-платформ...
4,YouTube,https://ru.wikipedia.org/wiki/YouTube,"YouTube (МФА: [ˈjuːtjuːb], «ютьюб», «ютюб», ча..."
5,NashStore,https://ru.wikipedia.org/wiki/NashStore,"NashStore — российский магазин приложений, раз..."
6,Инженер,https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D0%...,Инжене́р (фр. ingénieur ← от лат. ingenium — с...
7,Программист,https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%...,"Программи́ст — специалист, занимающийся програ..."
8,Escape from Tarkov,https://ru.wikipedia.org/wiki/Escape_from_Tarkov,Escape from Tarkov (с англ. — «Побег из Тáрков...
9,Ethereum,https://ru.wikipedia.org/wiki/Ethereum,"Ethereum (Эфириум, от англ. ether [ˈiːθə] — «э..."


In [269]:
# отранижируем рекомендованные на предыдущем шаги статьи
range_articles(query, articles['text'])

,article,frequency_rank,distance_rank,position_rank,rank
24,"Систе́ма управле́ния ба́зами да́нных, сокр. СУ...",0.02,0.2475,0.275,0.180833
12,"Загон — место на выгоне или пастбище, огорожен...",0.00641,0.192308,0.224359,0.141026
28,Согласованность данных (иногда консистентность...,0.071429,0.0625,0.232143,0.122024
16,БАЗ-8027 — двухосное полноприводное внедорожно...,0.020833,0.0625,0.25,0.111111
18,БАЗ-8029 — трёхосное шасси для установки авток...,0.019231,0.0625,0.25,0.110577
11,"Ба́за да́нных — совокупность данных, хранимых ...",0.032051,0.0625,0.230769,0.10844
21,"Ба́за да́нных — совокупность данных, хранимых ...",0.032051,0.0625,0.230769,0.10844
13,БАЗ-5937 — специальное трёхосное плавающее шас...,0.0125,0.0625,0.25,0.108333
14,БАЗ А079 «Эталон» — автобус малого класса для ...,0.011364,0.0625,0.25,0.107955
31,«Умные вещи» — советский музыкальный фильм-ска...,0.010417,0.0625,0.25,0.107639
